In [1]:
# defining functions to perform And(A^B), Not(~A), Or(AVB) and Implies((~A)VB)
def op_and(a, b):
    return a and b

def op_not(a):
    return not a

def op_or(a, b):
    return a or b

def op_implies(a, b):
    return op_or(op_not(a), b)

In [2]:
# evaluating final expression
operations = {'^': op_and, 'V': op_or, '=>': op_implies}

def evaluate_expression(exp, values):
  idx = 0
  lhs = ""
  rhs = ""
  operation = ""
  l = len(exp)

  if l == 1:  return values[exp[0]]
  elif l == 2:  return str((int(values[exp[1]]) + 1) % 2)

  while idx < l:
    if exp[idx] == '^' or exp[idx] == 'V' or exp[idx] == '=':
      operation = exp[idx]
      break
    lhs += exp[idx]
    idx += 1
  idx += 1

  if exp[idx] == '>':
    operation += exp[idx]
    idx += 1
  while idx < l:
    rhs += exp[idx]
    idx += 1

  # handling negation
  if lhs[0] == '~': left_value = values[lhs[1]]
  else: left_value = values[lhs[0]]
  if rhs[0] == '~': right_value = values[rhs[1]]
  else: right_value = values[rhs[0]]

  # performing operations
  result = operations[operation](int(left_value), int(right_value))
  return str(result)

In [3]:
# getting expression evaluated 
def simplify(query, values):
    stack = []
    for i, c in enumerate(query):
        if c == ")":
            exp = []
            while stack[-1] != "(":
                exp.append(stack.pop())
            #print(exp[::-1])
            stack.append(evaluate_expression(exp[::-1], values))
        else:
            stack.append(c)
    if stack[-1] == "1":
        return True
    return False

In [4]:
# main
if __name__ == "__main__":

  # taking input expression and finding out the number of variables in it
  input_expression = str(input("Enter the expression to determine whether it is a theorem: "))
  variable = set([x for x in input_expression if x not in ['=', '>', '~', ')', '(', '^', 'V']])
  truth_table = []
  truth_table_row = ""
  for v in variable:
    truth_table_row = truth_table_row + v + " "
  truth_table_row = truth_table_row + input_expression
  truth_table.append(truth_table_row)

  # initially flag will be false. False => not theorem
  flag = False

  # finding results corresponding to all the truth table values of the variables
  for i in range(2 ** len(variable)):
    
    # getting corresponding truth table values
    values = {'0': '0', '1': '1'}
    truth = format(i, '0' + str(len(variable)) + 'b')
    for i, c in enumerate(variable):
      values[c] = truth[i]

    # checking if for any True LHS, we are getting False RHS 
    stack = []
    index = 0
    for j, c in enumerate(input_expression):
      if c == ")":
        stack.pop()
        if len(stack) == 0:
          index = j
          break
      elif c == "(":
        stack.append(c)

    # working on LHS and RHS separately
    if "=>" in input_expression:
        lhs = input_expression[:index + 1]
        rhs = input_expression[index + 3:]
        # (p-> q ) =>(q)
        result_ans = True
        if simplify(lhs, values) and not simplify(rhs, values):
            flag = True
            result_ans = False  


        
        truth_table_row = ""
        for k in values.keys():
          if k != '0' and k != '1':
            truth_table_row = truth_table_row + values[k] + " "

        truth_table_row = truth_table_row + str(result_ans)
        truth_table.append(truth_table_row)

  # Results
  for r in truth_table:
    print(r)

  if flag:
    print("Result-> Not a Theorem!")
  else:
    print("Result-> Is a Theorem!")

# Given test cases + true cases:
# (P=>Q)=>((~Q=>P)=>Q)
# (P)=>(PVQ)
# (P^Q)=>(PVR)
# (b)=>(aVb)

# Some false test cases:
# (a)=>(b)
# (aVb)=>(b)
# (b)=>(aVc)
#((~r)Vp)=>(q)

Enter the expression to determine whether it is a theorem: (a)=>(b)
a b (a)=>(b)
0 0 True
0 1 True
1 0 False
1 1 True
Result-> Not a Theorem!
